In [1]:
import pandas as pd
from tqdm import tqdm
import tensorflow_datasets as tfds

In [2]:
questions = []
answers = []
dataset = []

dataset_name = list(tfds.builder('math_dataset').builder_configs.keys())[1]
builder = tfds.builder(f'math_dataset/{dataset_name}')

builder.download_and_prepare()
ds = builder.as_dataset(split='train', shuffle_files=True)
for example in tqdm(ds):
  questions.append(example['question'].numpy())
  answers.append(example['answer'].numpy())
  dataset.append(dataset_name)
df = pd.DataFrame({'question': questions, 'answer': answers, 'dataset': dataset})
df['question'] = [x.decode("utf-8") for x in df.question]
df['answer'] = [x.decode("utf-8") for x in df.answer]
df.to_csv("train.csv")
print(df.shape)
print(df.head())

Shuffling and writing examples to /root/tensorflow_datasets/math_dataset/algebra__linear_1d_composed/1.0.0.incompleteSJWQ1B/math_dataset-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/math_dataset/algebra__linear_1d_composed/1.0.0.incompleteSJWQ1B/math_dataset-test.tfrecord


Dataset math_dataset downloaded and prepared to /root/tensorflow_datasets/math_dataset/algebra__linear_1d_composed/1.0.0. Subsequent calls will reuse this data.


100%|██████████| 1999998/1999998 [08:23<00:00, 3972.89it/s]


(1999998, 3)
                                            question  ...                      dataset
0  Let u(r) = -2*r**2 - 38*r + 4. Let a be u(-19)...  ...  algebra__linear_1d_composed
1  Suppose -4*y = p - 20, -y - 5*p - 6 = 8. Let h...  ...  algebra__linear_1d_composed
2  Let g be (-23)/(-69) + (-527)/(-3). Let f = g ...  ...  algebra__linear_1d_composed
3  Suppose -5*d - 153 = -158. Let a(t) = t**2 - t...  ...  algebra__linear_1d_composed
4  Let b(c) = -2*c**2 - 33*c + 1295. Let i be b(1...  ...  algebra__linear_1d_composed

[5 rows x 3 columns]


In [5]:
questions = []
answers = []
dataset = []
dataset_name = list(tfds.builder('math_dataset').builder_configs.keys())[1]
builder = tfds.builder(f'math_dataset/{dataset_name}')

builder.download_and_prepare()
ds = builder.as_dataset(split='test', shuffle_files=True)
for example in tqdm(ds):
  questions.append(example['question'].numpy())
  answers.append(example['answer'].numpy())
  dataset.append(dataset_name)
df = pd.DataFrame({'question': questions, 'answer': answers, 'dataset': dataset})
df['question'] = [x.decode("utf-8") for x in df.question]
df['answer'] = [x.decode("utf-8") for x in df.answer]
df.to_csv("test.csv")
print(df.shape)
print(df.head())

100%|██████████| 10000/10000 [00:02<00:00, 3894.95it/s]

(10000, 3)
                                            question  ...                      dataset
0  Let m be -2 + 25/20*4. Suppose 0 = 4*t - 19 + ...  ...  algebra__linear_1d_composed
1  Suppose -d = 5 - 16. Let b = -579 + 584. Solve...  ...  algebra__linear_1d_composed
2  Let w be (-39)/((-1 + -5)/12). Solve -70*x = -...  ...  algebra__linear_1d_composed
3  Suppose 3*v - l + 9 = 4*v, 0 = -5*v + 5*l - 5....  ...  algebra__linear_1d_composed
4  Let w be 2 - (0 - 0)/(-2). Let f = -110 - -110...  ...  algebra__linear_1d_composed

[5 rows x 3 columns]


In [2]:
from utils import install_dependencies, set_seed, print_items_from_dict

In [3]:
package_names = ["transformers", "nlp", "sentencepiece", "pytorch_lightning==0.9", "rouge_score"]
install_dependencies(package_names)

1. pip install transformers
2. pip install nlp
3. pip install sentencepiece
4. pip install pytorch_lightning==0.9
5. pip install rouge_score



In [4]:
set_seed(42)

In [5]:
import json
import argparse
from nlp import load_metric
from model import T5FineTuner
import pytorch_lightning as pl
from dataset import get_dataset
from transformers import T5Tokenizer
from callback import LoggingCallback, logger

In [6]:
with open('config.json') as f:
  args_dict = json.load(f)
args = argparse.Namespace(**args_dict)
print_items_from_dict(args_dict)

Items held:
output_dir (t5_math_problems)
model_name (t5-small)
tokenizer_name (t5-small)
max_input_length (140)
max_output_length (10)
freeze_encoder (False)
freeze_embeds (False)
learning_rate (0.0003)
weight_decay (0.0)
adam_epsilon (1e-08)
warmup_steps (0)
train_batch_size (128)
eval_batch_size (128)
num_train_epochs (5)
gradient_accumulation_steps (1)
n_gpu (1)
resume_from_checkpoint (None)
val_check_interval (0.8)
n_val (1000)
n_train (-1)
n_test (-1)
early_stop_callback (False)
fp_16 (False)
opt_level (O1)
max_grad_norm (1.0)
seed (42)
automatic_optimization (False)


In [7]:
## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=3
)

## If resuming from checkpoint, add an arg resume_from_checkpoint
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    val_check_interval=args.val_check_interval,
    logger=None,
    callbacks=[LoggingCallback()],
)

In [8]:
model = T5FineTuner(args)

In [9]:
trainer = pl.Trainer(**train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [10]:
trainer.fit(model)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60 M  


Saving latest checkpoint..


1